In [1]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model


     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 6.4 MB 19.2 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 46 kB 3.0 MB/s 


In [2]:
clmns = [
    "Open 12-11","Open 11-10","Open 10-9","Open 9-8","Open 8-7","Open 7-6","Open 6-5","Open 5-4","Open 4-3","Open 3-2","Open 2-1",
    "Close 12-11","Close 11-10","Close 10-9","Close 9-8","Close 8-7","Close 7-6","Close 6-5","Close 5-4","Close 4-3","Close 3-2","Close 2-1",
    "High 12-11","High 11-10","High 10-9","High 9-8","High 8-7","High 7-6","High 6-5","High 5-4","High 4-3","High 3-2","High 2-1",
    "Low 12-11","Low 11-10","Low 10-9","Low 9-8","Low 8-7","Low 7-6","Low 6-5","Low 5-4","Low 4-3","Low 3-2","Low 2-1",
    "AdjV 12-11","AdjV 11-10","AdjV 10-9","AdjV 9-8","AdjV 8-7","AdjV 7-6","AdjV 6-5","AdjV 5-4","AdjV 4-3","AdjV 3-2","AdjV 2-1",
    "Volume 12-11","Volume 11-10","Volume 10-9","Volume 9-8","Volume 8-7","Volume 7-6","Volume 6-5","Volume 5-4","Volume 4-3","Volume 3-2","Volume 2-1",
    "suggestion"
]

def read_syms_from_txt():
  with open("syms.txt", "r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["yes","no"], axis=1)
  y = data[["yes","no"]]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def scaler(row):
    scaler = MinMaxScaler(feature_range=(-3, 3))
    row = scaler.fit_transform(row)
    return row
def process(data):
    data = data.dropna()
    row = []
    data = np.array(data)
    llst = [0, 1, 2, 3, 4, 5]

    for i in range(12, data.shape[0]):
        grow = []
        srow = []

        for j in llst:

           srow.append([
               data[i-1][j] - data[i-2][j],
               data[i-2][j] - data[i-3][j],
               data[i-3][j] - data[i-4][j],
               data[i-4][j] - data[i-5][j],
               data[i-5][j] - data[i-6][j],
               data[i-6][j] - data[i-7][j],
               data[i-7][j] - data[i-8][j],
               data[i-8][j] - data[i-9][j],
               data[i-9][j] - data[i-10][j],
               data[i-10][j] - data[i-11][j],
               data[i-11][j] - data[i-12][j]
           ])

        for lst in srow:
            mm = np.array(lst)
            mm = np.reshape(mm, (-1, 1))
            grow.append(scaler(mm))

        sugg = "no"
        if data[i][3] > data[i][0]:
            sugg = "yes"

        arr = np.array(grow).flatten()
        arr = np.append(arr, sugg)
        row.append(arr)


    grow = []
    srow = []
    llst = []
    data = []
    arr = []
    mm = []

    return np.array(row)

In [ ]:
def get_data(symbols):
    unattached_dfs = []
    for symbol in symbols:
         data = yf.download(symbol, period="max",
                            interval="1d", progress=False)
         if data.empty:
              pass
         else:
               if data.shape[0] > 12:
                   unattached_dfs.append(
                       process(data))

    data = np.array(unattached_dfs[0])
    for i in unattached_dfs[1:]:
        data = np.append(data, np.array(i), axis=0)
        
    symbol = []

    data = pd.DataFrame(data, columns=clmns)
    sugg = data["suggestion"]
    data.drop("suggestion",axis=1,inplace=True)
    sugg = pd.get_dummies(sugg)
    data = pd.concat([data,sugg],axis=1)
    data = data.astype(float)

    xTrain, xTest, yTrain, yTest = spliting(data)
    print(data.shape)
    data.to_csv(f"{random.randint(1,10000)}.csv")
    #data = []
    return xTrain, xTest, yTrain, yTest,data



#symbols = read_syms_from_txt()
symbols = get_crypto_syms()

pieces = 15
new_arrays = np.array_split(symbols, pieces)
for symbols in new_arrays[:7]:
    get_data(symbols)
#symbols = ["btc-usd","eth-usd","trx-usd","bnb-usd","xrp-usd"]
#xTrain, xTest, yTrain, yTest,data= get_data(symbols)
#data 

In [4]:
data = pd.read_parquet("/content/drive/MyDrive/cd.parquet")

In [5]:
data

,Open 12-11,Open 11-10,Open 10-9,Open 9-8,Open 8-7,Open 7-6,Open 6-5,Open 5-4,Open 4-3,Open 3-2,...,Volume 9-8,Volume 8-7,Volume 7-6,Volume 6-5,Volume 5-4,Volume 4-3,Volume 3-2,Volume 2-1,no,yes
0,-3.000000,-1.957569,-0.837604,-2.659657,0.462580,-1.236956,3.000000,0.025922,-1.290769,-1.306116,...,0.519937,0.930820,-0.347221,0.297778,3.000000,0.776289,0.884565,-3.000000,1.0,0.0
1,-2.443146,-3.000000,-1.957569,-0.837604,-2.659657,0.462580,-1.236956,3.000000,0.025922,-1.290769,...,-0.327512,-1.445592,-0.709071,-3.000000,-1.843818,3.000000,-0.986073,-0.791985,1.0,0.0
2,-3.000000,-2.170767,-2.699756,-1.709489,-0.645568,-2.376444,0.589555,-1.024936,3.000000,0.174747,...,-1.137965,-0.327512,-1.445592,-0.709071,-3.000000,-1.843818,3.000000,-0.986073,1.0,0.0
3,-3.000000,-2.150761,-1.438897,-1.893013,-1.042908,-0.129574,-1.615462,0.930729,-0.455247,3.000000,...,0.011597,-1.137965,-0.327512,-1.445592,-0.709071,-3.000000,-1.843818,3.000000,0.0,1.0
4,-0.957364,-3.000000,-2.150761,-1.438897,-1.893013,-1.042908,-0.129574,-1.615462,0.930729,-0.455247,...,-1.704639,-0.535872,-1.476459,-0.813335,-1.728162,-1.125532,-3.000000,-2.053997,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121540,-0.227182,0.863752,2.195289,0.876397,2.415152,2.511538,-3.000000,1.038192,2.191452,0.514726,...,2.902718,-0.405244,-0.107661,0.441227,0.799626,-0.820675,-1.889915,3.000000,0.0,1.0
121541,2.437672,0.018560,1.206179,2.655723,1.219944,2.895071,3.000000,-3.000000,1.396078,2.651546,...,-3.000000,3.000000,-0.362480,-0.059993,0.497941,0.862248,-0.784758,-1.871620,1.0,0.0
121542,3.000000,1.916706,-0.270639,0.803198,2.113867,0.815645,2.330284,2.425160,-3.000000,0.974904,...,0.431850,-3.000000,3.000000,-0.362480,-0.059993,0.497941,0.862248,-0.784758,1.0,0.0
121543,-0.149419,3.000000,1.916706,-0.270639,0.803198,2.113867,0.815645,2.330284,2.425160,-3.000000,...,0.818034,0.431850,-3.000000,3.000000,-0.362480,-0.059993,0.497941,0.862248,0.0,1.0


In [7]:
xTrain, xTest, yTrain, yTest = spliting(data)


(1603365, 66) (1603365, 2)
(178152, 66) (178152, 2)


In [11]:
model = Sequential()



model.add(Dense(2000, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(1500, activation='relu'))
model.add(Dense(1500, activation='relu'))
model.add(Dense(1500, activation='relu'))
model.add(Dense(1500, activation='relu'))
model.add(Dense(1500, activation='relu'))
model.add(Dense(1750, activation='relu'))
model.add(Dense(2000, activation='relu'))
model.add(Dense(2500, activation='relu'))



model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer="adam", loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 2000)              134000    
                                                                 
 dense_19 (Dense)            (None, 1500)              3001500   
                                                                 
 dense_20 (Dense)            (None, 1500)              2251500   
                                                                 
 dense_21 (Dense)            (None, 1500)              2251500   
                                                                 
 dense_22 (Dense)            (None, 1500)              2251500   
                                                                 
 dense_23 (Dense)            (None, 1500)              2251500   
                                                                 
 dense_24 (Dense)            (None, 1750)             

In [12]:
model.fit(xTrain,yTrain,epochs=30,batch_size=10000,validation_data=(xTest,yTest))

Epoch 1/30
161/161 [==============================] - 60s 371ms/step - loss: 0.6811 - accuracy: 0.5553 - val_loss: 0.6550 - val_accuracy: 0.5926
Epoch 2/30
161/161 [==============================] - 60s 374ms/step - loss: 0.6305 - accuracy: 0.6248 - val_loss: 0.6110 - val_accuracy: 0.6452
Epoch 3/30
161/161 [==============================] - 61s 377ms/step - loss: 0.5768 - accuracy: 0.6760 - val_loss: 0.5544 - val_accuracy: 0.6974
Epoch 4/30
161/161 [==============================] - 61s 378ms/step - loss: 0.4931 - accuracy: 0.7482 - val_loss: 0.4658 - val_accuracy: 0.7720
Epoch 5/30
161/161 [==============================] - 61s 378ms/step - loss: 0.3644 - accuracy: 0.8372 - val_loss: 0.3562 - val_accuracy: 0.8490
Epoch 6/30
161/161 [==============================] - 61s 379ms/step - loss: 0.2486 - accuracy: 0.9014 - val_loss: 0.2693 - val_accuracy: 0.8989
Epoch 7/30
161/161 [==============================] - 61s 379ms/step - loss: 0.1791 - accuracy: 0.9343 - val_loss: 0.2319 - val_ac

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive
